In [1]:
import dataLoader as loader
import HDDMmodelMaker as ddm
import pandas as pd
import time
import matplotlib.pyplot as plt
import hddm
import random
import scipy.stats as st
import numpy as np
import seaborn as sns
import pymc as mc
import random 
import kabuki
from mpl_toolkits.mplot3d import Axes3D
import statsmodels
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial import distance

/Users/eladiomontero/opt/anaconda3/envs/decTimesEnv/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


In [2]:
data = hddm.load_csv("data_actions.csv")

In [3]:
def actionChange(actions1, actions2, user1, user2, rt):
    if len(actions1) != len(actions2): return "Lenghts mismatch" #Raise exception
    rounds = len(actions1)
    prev1 = ""
    prev2 = ""
    switch_events = pd.DataFrame(columns=["type", "lag", "subject", "round", "rt"])
    lag = 0
    for r in range(0, rounds):
        if r ==0:
            prev1 = actions1[r]
            prev2 = actions2[r]
            lag += 1
            continue
        elif actions1[r] != prev1:
            switch_events = switch_events.append(
                {"context": prev2, "type": "%s%s" % (prev1, actions1[r]),
                 "lag": lag, "subject": user1, "round": r + 1, "rt": rt[r]}, ignore_index=True)
            lag = 0
        if actions2[r] == prev2:
            lag += 1
        else: lag = 1
        prev1 = actions1[r]
        prev2 = actions2[r]
    return switch_events

In [4]:
def actionCounts(actions, user):
    prosocial = ['CCC', 'CDC', '00C', 'DDC', 'DCC']
    proself = ['DDD', 'DCD', '00D', 'CCD', 'CDD']
    rating = 0
    act_df = {'user':user,'00D':0,'00C':0,'DCD':0,'CDC':0,'CCC':0,'CDD':0,'CCD':0,'DDC':0,'DCC':0,'DDD':0, 'type':"", 'score': 0}
    for a in actions:
        act_df[a] = act_df[a] + 1
        if a in prosocial:
            rating = rating + 1
        else: rating = rating - 1
    act_df["score"] = rating
    if rating >= 0: act_df["type"] = "Pro-social"
    else: act_df["type"] = "Pro-self"
    return act_df

In [5]:
switches = pd.DataFrame()
actions = pd.DataFrame()
for t in data.treatment.unique():
    t_df = data.loc[(data.treatment == t)]
    users = t_df.player.unique()
    switchesT = pd.DataFrame()
    actionsT = pd.DataFrame()
    for u in users:
        df = t_df.loc[t_df.player == u]
        df = df.sort_values(by = "round", ascending = True)
        opp = str(df.opponent.unique())
        acc1 = df.action_player.values
        acc2 = df.action_opponent.values
        rt = df["rt"].values
        switchesT = switchesT.append(actionChange(acc1, acc2, u, opp, rt))
        actionsT = actionsT.append(actionCounts(df.actions, u), ignore_index=True)
    switchesT["treatment"] = t
    actionsT["treatment"] = t
    switches = switches.append(switchesT, ignore_index = True)
    actions = actions.append(actionsT, ignore_index = True)

/Users/eladiomontero/opt/anaconda3/envs/decTimesEnv/lib/python2.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [6]:
## Adjusting weights for actions
actions["type_adj"] = ""
actions["score_adj"] = 0

for index, row in actions.iterrows():
    score = sum(row[0:10] * [4,-4,0,-4,4,0,0,-4,4,0])
    actions.iloc[index, 15] = score
    actions.iloc[index, 14] = "Pro-social" if score>=0 else "Pro-self"
actions

,00C,00D,CCC,CCD,CDC,CDD,DCC,DCD,DDC,DDD,score,type,user,treatment,type_adj,score_adj
0,0.0,1.0,69.0,9.0,1.0,2.0,9.0,1.0,3.0,5.0,64.0,Pro-social,s3m7_usuario9,weak,Pro-self,-28.0
1,1.0,0.0,74.0,4.0,9.0,1.0,2.0,1.0,3.0,5.0,78.0,Pro-social,s3m7_usuario18,weak,Pro-social,32.0
2,1.0,0.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,Pro-social,s3m7_usuario7,weak,Pro-social,4.0
3,1.0,0.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,Pro-social,s3m7_usuario10,weak,Pro-social,4.0
4,1.0,0.0,97.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,98.0,Pro-social,s3m7_usuario13,weak,Pro-social,0.0
5,1.0,0.0,98.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,100.0,Pro-social,s3m7_usuario5,weak,Pro-social,8.0
6,0.0,1.0,93.0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,90.0,Pro-social,s3m7_usuario15,weak,Pro-social,0.0
7,1.0,0.0,93.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,90.0,Pro-social,s3m7_usuario16,weak,Pro-social,4.0
8,0.0,1.0,86.0,2.0,1.0,1.0,3.0,3.0,1.0,2.0,82.0,Pro-social,s3m7_usuario12,weak,Pro-self,-16.0
9,1.0,0.0,88.0,0.0,3.0,3.0,1.0,1.0,2.0,1.0,90.0,Pro-social,s3m7_usuario6,weak,Pro-social,20.0


In [7]:
data_svo = pd.merge(data, actions[["type", "user"]], left_on = "player", right_on = "user")
data_svo.opponent = data_svo['opponent'].str.strip("['']")

In [8]:
data_svo

,Unnamed: 0,round,player,action_player,opponent,action_opponent,payoff,rt,session,treatment,actions,response,abs_time,type,user
0,0,1,s3m7_usuario9,D,s3m7_usuario18,C,4,-4.452,s3m7,weak,00D,0,4.452,Pro-social,s3m7_usuario9
1,18,2,s3m7_usuario9,C,s3m7_usuario18,C,3,5.401,s3m7,weak,DCC,1,5.401,Pro-social,s3m7_usuario9
2,36,3,s3m7_usuario9,D,s3m7_usuario18,C,4,-10.751,s3m7,weak,CCD,0,10.751,Pro-social,s3m7_usuario9
3,54,4,s3m7_usuario9,C,s3m7_usuario18,C,3,3.443,s3m7,weak,DCC,1,3.443,Pro-social,s3m7_usuario9
4,72,5,s3m7_usuario9,C,s3m7_usuario18,C,3,6.401,s3m7,weak,CCC,1,6.401,Pro-social,s3m7_usuario9
5,90,6,s3m7_usuario9,D,s3m7_usuario18,C,4,-5.265,s3m7,weak,CCD,0,5.265,Pro-social,s3m7_usuario9
6,108,7,s3m7_usuario9,C,s3m7_usuario18,D,0,5.641,s3m7,weak,DCC,1,5.641,Pro-social,s3m7_usuario9
7,126,8,s3m7_usuario9,D,s3m7_usuario18,C,4,-10.555,s3m7,weak,CDD,0,10.555,Pro-social,s3m7_usuario9
8,144,9,s3m7_usuario9,C,s3m7_usuario18,C,3,10.590,s3m7,weak,DCC,1,10.590,Pro-social,s3m7_usuario9
9,162,10,s3m7_usuario9,D,s3m7_usuario18,D,0,-7.549,s3m7,weak,CCD,0,7.549,Pro-social,s3m7_usuario9


# weak

In [11]:
weak = data_svo[data_svo.treatment == "weak"]
weak["type"].value_counts()

Pro-social    1800
Name: type, dtype: int64

In [12]:
## trying a model per subject, with the whole dataset, then partitioning afterwards

params_weak = actions.loc[actions.treatment == "weak", ["user", "type", "score", "type_adj", "score_adj"]]
params_weak["a"] = 0
params_weak["v"] = 0
params_weak["t"] = 0
params_weak["z"] = 0
params_weak

modeler = ddm.HDDMmodelMaker()
for s in weak.player.unique():
    model_soc_s3 = modeler.fit_model(weak.loc[(weak["player"] == s)], 5000, 200)
    params_weak.loc[params_weak.user == s, ["a","v","t","z"]] = model_soc_s3.gen_stats()["mean"].values
    

 [-----------------100%-----------------] 5000 of 5000 complete in 5.1 sec

In [18]:
params_weak["y"] = 0
params_weak["y_adj"] = 0
params_weak.loc[params_weak["type"] == "Pro-social", ["y"]] = 1
params_weak.loc[params_weak["type"] == "Pro-self", ["y"]] = 0
params_weak.loc[params_weak["type_adj"] == "Pro-social", ["y_adj"]] = 1
params_weak.loc[params_weak["type_adj"] == "Pro-self", ["y_adj"]] = 0
weak_df = params_weak[["a","v", "t", "z", "y_adj"]]
weak_df

,a,v,t,z,y_adj
0,3.255287,0.104549,0.349740,0.761625,0
1,2.709906,0.175382,0.377830,0.830882,1
2,2.047376,2.015130,0.354075,0.725827,1
3,4.169979,0.461699,0.346234,0.885307,1
4,2.965683,0.841667,0.380724,0.775296,1
5,2.666073,1.158333,0.359542,0.738211,1
6,3.752886,0.534211,0.146685,0.711207,1
7,2.326765,0.926423,0.302067,0.701246,1
8,4.010591,0.105384,0.374135,0.860828,0
9,2.910864,0.446095,0.263144,0.824835,1


In [25]:
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(weak_df[["a","v", "t", "z"]], weak_df[["y_adj"]], test_size=0.3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape


(12, 4) (12, 1)
(6, 4) (6, 1)


In [27]:
## Test with logistic regression
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
model = LogisticRegression()

In [28]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

# Perform 6-fold cross validation
scores = cross_val_score(model, weak_df[["a","v", "t", "z"]], weak_df[["y_adj"]], cv=10)
print ("Cross-validated scores:", scores)

('Cross-validated scores:', array([0.66666667, 0.66666667, 0.66666667, 0.5       , 0.5       ,
       1.        , 1.        , 1.        , 1.        , 1.        ]))


In [ ]:
from sklearn import svm
clf = svm.SVC()

# Perform 6-fold cross validation
scores = cross_val_score(clf, weak_df[["a","v", "t", "z"]], weak_df[["y"]], cv=10)
print ("Cross-validated scores:", scores)

In [31]:
rounds_windows = [10, 20, 30, 40, 50, 60, 70, 80, 90]

for w in rounds_windows:
    params_weak["a%d" % w] = 0
    params_weak["v%d" % w] = 0
    params_weak["t%d" % w] = 0
    params_weak["z%d" % w] = 0
    modeler = ddm.HDDMmodelMaker()
    for s in weak.player.unique():
        model_soc_s3 = modeler.fit_model(weak.loc[(weak["player"] == s) & (weak["round"] <= w)], 5000, 200)
        params_weak.loc[params_weak.user == s, ["a%d" % w,"v%d" % w,"t%d" % w,"z%d" % w]] = model_soc_s3.gen_stats()["mean"].values

 [-----------------100%-----------------] 5000 of 5000 complete in 5.5 secc

In [32]:
params_weak

,user,type,score,type_adj,score_adj,a,v,t,z,y,...,t70,z70,a80,v80,t80,z80,a90,v90,t90,z90
0,s3m7_usuario9,Pro-social,64.0,Pro-self,-28.0,3.255287,0.104549,0.349740,0.761625,1,...,0.401156,0.647393,3.353873,0.130088,0.319666,0.689643,3.303895,0.120307,0.319910,0.726400
1,s3m7_usuario18,Pro-social,78.0,Pro-social,32.0,2.709906,0.175382,0.377830,0.830882,1,...,0.341273,0.766892,2.752182,0.178919,0.356511,0.787959,2.734625,0.128595,0.374955,0.831497
2,s3m7_usuario7,Pro-social,100.0,Pro-social,4.0,2.047376,2.015130,0.354075,0.725827,1,...,0.347706,0.694294,2.087110,1.938411,0.352517,0.702009,2.086342,2.004799,0.357919,0.714908
3,s3m7_usuario10,Pro-social,100.0,Pro-social,4.0,4.169979,0.461699,0.346234,0.885307,1,...,0.361081,0.885163,4.147433,0.402591,0.363191,0.887169,4.141283,0.418708,0.366386,0.891226
4,s3m7_usuario13,Pro-social,98.0,Pro-social,0.0,2.965683,0.841667,0.380724,0.775296,1,...,0.354430,0.732885,2.989684,0.791543,0.358643,0.751472,2.992355,0.804553,0.374189,0.771599
5,s3m7_usuario5,Pro-social,100.0,Pro-social,8.0,2.666073,1.158333,0.359542,0.738211,1,...,0.321639,0.698982,2.700044,1.151309,0.337738,0.708775,2.700737,1.163947,0.346512,0.724151
6,s3m7_usuario15,Pro-social,90.0,Pro-social,0.0,3.752886,0.534211,0.146685,0.711207,1,...,0.130610,0.708282,3.785366,0.442885,0.142059,0.721648,3.763010,0.495295,0.145357,0.716669
7,s3m7_usuario16,Pro-social,90.0,Pro-social,4.0,2.326765,0.926423,0.302067,0.701246,1,...,0.286604,0.696679,2.361859,0.795327,0.294346,0.702540,2.349652,0.891381,0.295250,0.693088
8,s3m7_usuario12,Pro-social,82.0,Pro-self,-16.0,4.010591,0.105384,0.374135,0.860828,1,...,0.357604,0.849113,4.059492,0.044933,0.374303,0.864483,4.039857,0.085280,0.373667,0.859247
9,s3m7_usuario6,Pro-social,90.0,Pro-social,20.0,2.910864,0.446095,0.263144,0.824835,1,...,0.266462,0.793692,2.885793,0.396165,0.276592,0.811327,2.891852,0.423034,0.269452,0.821919


In [31]:
for r in rounds_windows:
    X, y = params_weak[["a%d" % r,"v%d" % r, "t%d" % r, "z%d" % r]], params_weak[["y"]]
    model = LogisticRegression()
    scores = cross_val_score(model, X, y, cv=10)
    print ("Cross-validated scores %d: %f, %f" % (r,np.mean(scores), np.std(scores)))

Cross-validated scores 10: 0.802727, 0.070557
Cross-validated scores 20: 0.802727, 0.050091
Cross-validated scores 30: 0.822929, 0.047283
Cross-validated scores 40: 0.834040, 0.048409
Cross-validated scores 50: 0.822929, 0.047283
Cross-validated scores 60: 0.832929, 0.051741
Cross-validated scores 70: 0.832929, 0.051741
Cross-validated scores 80: 0.865152, 0.063231
Cross-validated scores 90: 0.865152, 0.080420


In [33]:
for r in rounds_windows:
    X, y = params_weak[["a%d" % r,"v%d" % r, "t%d" % r, "z%d" % r]], params_weak[["y_adj"]]
    model = LogisticRegression()
    scores = cross_val_score(model, X, y, cv=10)
    print ("Cross-validated scores %d: %f, %f" % (r,np.mean(scores), np.std(scores)))

Cross-validated scores 10: 0.800000, 0.208167
Cross-validated scores 20: 0.800000, 0.208167
Cross-validated scores 30: 0.700000, 0.305505
Cross-validated scores 40: 0.800000, 0.208167
Cross-validated scores 50: 0.800000, 0.208167
Cross-validated scores 60: 0.800000, 0.208167
Cross-validated scores 70: 0.800000, 0.208167
Cross-validated scores 80: 0.800000, 0.208167
Cross-validated scores 90: 0.800000, 0.208167


In [34]:
params_weak.to_csv("params_weak.csv")